## Importar librerias y definición de la ruta  de trabajo (path)

In [58]:

import pandas as pd
import numpy as np
import os
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

%matplotlib inline

# Tratamiento de datos
# ==============================================================================
import shap
from skforecast.utils import load_forecaster
from skforecast.utils import save_forecaster
from skforecast.preprocessing import RollingFeatures
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import TimeSeriesFold
from skforecast.direct import ForecasterDirect
from skforecast.recursive import ForecasterRecursive
import skforecast
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import sklearn
from skforecast.datasets import fetch_dataset

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['font.size'] = 10

# Modelado y Forecasting
# ==============================================================================


# Configuración warnings
# ==============================================================================
warnings.filterwarnings('once')

color = '\033[1m\033[38;5;208m'
print(f"{color}Versión skforecast: {skforecast.__version__}")
print(f"{color}Versión scikit-learn: {sklearn.__version__}")
print(f"{color}Versión pandas: {pd.__version__}")
print(f"{color}Versión numpy: {np.__version__}")

# Formato de los prints
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def headr(text):
    return ('\n'+color.UNDERLINE + text + color.END+'\n')


Versión skforecast: 0.15.1
Versión scikit-learn: 1.6.1
Versión pandas: 2.2.3
Versión numpy: 1.26.4


In [2]:
# Ingresa la ruta donde está el repositorio
ruta = 'c:/repo_remoto/'

## Integración

### Carga de archivos

In [ ]:
### Características Equipos

equipos = pd.read_csv(ruta + 'etapa4/data/Caracteristicas_Equipos.csv')
equipos_df = pd.DataFrame(equipos)
### Historicos Ordenes

ordenes = pd.read_csv(ruta + 'etapa4/data/Historicos_Ordenes.csv')
ordenes_df = pd.DataFrame(ordenes)
### Registros Condiciones

condiciones = pd.read_csv(ruta + 'etapa4/data/Registros_Condiciones.csv')
condiciones_df = pd.DataFrame(condiciones)

### Tratar equipos_df

In [ ]:
equipos_df

In [ ]:
# Mostrar el número de filas duplicadas
print("Número de filas duplicadas:", equipos_df.duplicated().sum())

# Mostrar todas las filas duplicadas considerando el índice como parte de los datos
duplicados = equipos_df[equipos_df.duplicated(keep=False)]
duplicados

### Tratar ordenes_df

In [ ]:
ordenes_df

In [ ]:
# Elimino ID_Orden, establezco Fecha como índice y le doy formato datetime
ordenes_df = ordenes_df.drop(columns=['ID_Orden']).set_index('Fecha')
ordenes_df.index = pd.to_datetime(ordenes_df.index)

In [ ]:
# Mostrar el número de filas duplicadas considerando el índice como parte de los datos
print("Número de filas duplicadas (incluyendo índice como columna):", ordenes_df.reset_index().duplicated(keep=False).sum())

# Mostrar todas las filas duplicadas considerando el índice como parte de los datos
duplicados = ordenes_df.reset_index()[ordenes_df.reset_index().duplicated(keep=False)]

### Tratar condiciones_df

In [ ]:
condiciones_df

In [ ]:
# Elimino ID_Registro, establezco Fecha como índice y le doy formato datetime
condiciones_df = condiciones_df.drop(columns=['ID_Registro']).set_index('Fecha')
condiciones_df.index = pd.to_datetime(condiciones_df.index)

In [ ]:
# Mostrar el número de filas duplicadas considerando el índice como parte de los datos
print("Número de filas duplicadas (incluyendo índice como columna):", condiciones_df.reset_index().duplicated(keep=False).sum())

# Mostrar todas las filas duplicadas considerando el índice como parte de los datos
duplicados = condiciones_df.reset_index()[condiciones_df.reset_index().duplicated(keep=False)]

### Merge

In [ ]:
# Asegurarse de que los índices sean de tipo datetime
ordenes_df.index = pd.to_datetime(ordenes_df.index)
condiciones_df.index = pd.to_datetime(condiciones_df.index)

# Mezclar los DataFrames basándose en el índice (fechas) y la columna 'ID_Equipo'
merged_df = pd.merge(
    ordenes_df.reset_index(),  # Reseteamos el índice temporal para incluirlo como columna
    condiciones_df.reset_index(),
    on=['Fecha', 'ID_Equipo'],  # Mezclamos por la fecha y el ID_Equipo
    how='outer'
)

# Volver a establecer 'Fecha' como índice
merged_df.set_index('Fecha', inplace=True)

# Mostrar el resultado
print(merged_df.info())

In [ ]:
# Relleno la columna Tipo_Mantenimiento vacías con Preventivo
merged_df['Tipo_Mantenimiento'] = merged_df['Tipo_Mantenimiento'].fillna('Preventivo')

In [ ]:
# Crear un diccionario con las ubicaciones conocidas por ID_Equipo
ubicaciones_por_equipo = ordenes_df[['ID_Equipo', 'Ubicacion']].dropna().drop_duplicates().set_index('ID_Equipo')['Ubicacion'].to_dict()

# Rellenar las filas vacías de Ubicacion en merged_df usando el diccionario
merged_df['Ubicacion'] = merged_df.apply(
    lambda row: ubicaciones_por_equipo.get(row['ID_Equipo'], row['Ubicacion']),
    axis=1
)

# Verificar si quedan valores vacíos en la columna Ubicacion
print("Valores vacíos restantes en 'Ubicacion':", merged_df['Ubicacion'].isna().sum())

In [ ]:
# Elimino columnas Costo_Mantenimiento y Duración_Horas
merged_df = merged_df.drop(columns=['Costo_Mantenimiento', 'Duracion_Horas'])

In [ ]:
# Combinar merged_df con equipos_df usando ID_Equipo como clave
merged_df = pd.merge(
    merged_df.reset_index(),  # Reseteamos el índice temporal para incluirlo como columna
    equipos_df,               # Información de los equipos
    on='ID_Equipo',           # Clave de unión
    how='left'                # Unión izquierda para mantener todas las filas de merged_df
)

# Volver a establecer 'Fecha' como índice
merged_df.set_index('Fecha', inplace=True)


In [ ]:
# Mostrar el resultado
merged_df

In [ ]:
# #Extraer el DataFrame unido a un archivo CSV

# merged_df = pd.DataFrame(merged_df)

# merged_df.to_csv(ruta + 'etapa4/output/merge_df.csv', index=True)


# print("El DataFrame merged_df se ha extraído a *.csv")

## Analisis y exportación Profiling

In [ ]:
merged_df = pd.read_csv(ruta + 'Etapa4/output/merge_df.csv')

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(merged_df, title="Mantenimiento Profiling Report")

In [ ]:
profile.to_notebook_iframe()

In [ ]:
profile.to_file(ruta + 'Etapa4/output/Mantenimiento.html')

## Resumen del Notebook


Este notebook realiza un análisis exhaustivo de datos relacionados con el mantenimiento de equipos, abarcando desde la carga y limpieza de datos hasta la generación de reportes de análisis. A continuación, se detalla cada sección:

---

1. **Carga y preparación de datos**:
   - Se importan librerías necesarias y se definen rutas de trabajo.
   - Se cargan tres conjuntos de datos: características de equipos, históricos de órdenes y registros de condiciones.
   - Se procesan los datos eliminando columnas irrelevantes, convirtiendo índices a formato de fecha y combinando los DataFrames en uno único (`merged_df`).

2. **Limpieza de datos**:
   - Se identifican y eliminan duplicados.
   - Se rellenan valores faltantes en columnas clave como `Tipo_Mantenimiento` y `Ubicacion`.
   - Se eliminan columnas innecesarias y se verifican posibles inconsistencias.

3. **Integración y análisis**:
   - Los datos se integran en un único DataFrame (`merged_df`) que combina información de equipos, órdenes y condiciones.
   - Se genera un reporte de análisis exploratorio utilizando `ydata_profiling` para identificar patrones, valores atípicos y distribuciones.

4. **Exportación de resultados**:
   - El DataFrame final se exporta a un archivo CSV para su uso posterior.
   - Se genera un reporte HTML con el análisis exploratorio.
---
#### Conclusión Final

El notebook proporciona un flujo completo para el análisis de datos de mantenimiento, desde la preparación e integración de datos hasta la generación de reportes detallados. Este proceso permite identificar patrones clave en los datos y preparar un conjunto limpio y estructurado para futuros análisis o modelado predictivo. La metodología aplicada asegura la calidad de los datos y facilita la toma de decisiones basada en información confiable.